In [1]:
from validphys.api import API
import numpy as np
import pandas as pd

fitname = "htcovmatfit"

In [2]:
ht_coeff = API.fit(fit=fitname).as_input()["theorycovmatconfig"]["ht_coeff"]

# dict used to produce theory predictions to construct the theory covmat as well as to produce 
# theory predictions from the fit performed using the ht covmat (i.e. the predicitons that should
# be compared to data)
common_dict = dict(
    dataset_inputs={"from_": "fit"},
    fit=fitname,
    fits=[fitname],
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
    theory={"from_": "fit"},
    theoryid={"from_": "theory"},
)

In [3]:
# Calculate theory predictions of the input PDF - this is used to construct the ht covmat
S_dict = dict(
    theorycovmatconfig={"from_": "fit"},
    pdf={"from_": "theorycovmatconfig"},
    use_t0=True,
    datacuts={"from_": "fit"},
    t0pdfset={"from_": "datacuts"},
)
preds_ht_cov_construction = API.group_result_table_no_table(**(S_dict|common_dict))

LHAPDF 6.5.0 loading /home/roy/miniconda3/envs/nnpdf/share/LHAPDF/210718-n3fit-data-003/210718-n3fit-data-003_0000.dat
210718-n3fit-data-003 PDF set, member #0, version 1
LHAPDF 6.5.0 loading all 101 PDFs in set 210718-n3fit-data-003
210718-n3fit-data-003, version 1; 101 PDF members


In [4]:
# Calculate theory predictions of the fit with ht covmat - this will be compared to data
# preds = API.group_result_table_no_table(pdf={"from_": "fit"}, **common_dict)

In [5]:
process_info = API.combine_by_type_ht(**(S_dict|common_dict))
kin_table = process_info.data
kin_df = pd.concat([pd.DataFrame(v) for v in kin_table.values()])
dsindex = API.groups_index(**common_dict)
kin_df = pd.DataFrame(kin_df.values, index=dsindex, columns=('kin1', 'kin2', 'kin3'))

In [35]:
dsindex

MultiIndex([('DIS NC',   'HERACOMBNCEP920', 108),
            ('DIS NC',   'HERACOMBNCEP920', 109),
            ('DIS NC',   'HERACOMBNCEP920', 110),
            ('DIS NC',   'HERACOMBNCEP920', 111),
            ('DIS NC',   'HERACOMBNCEP920', 112),
            ('DIS NC',   'HERACOMBNCEP920', 113),
            ('DIS NC',   'HERACOMBNCEP920', 114),
            ('DIS NC',   'HERACOMBNCEP920', 115),
            ('DIS NC',   'HERACOMBNCEP920', 116),
            ('DIS NC',   'HERACOMBNCEP920', 117),
            ...
            ('DIS CC', 'CHORUSNUPb_dw_ite', 596),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 597),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 598),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 599),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 600),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 601),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 602),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 604),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 605),
            ('DIS CC', 'CHORUSNUPb

In [34]:
common_dict = dict(
    dataset_inputs={"from_": "fit"},
    fit=fitname,
    fits=[fitname],
    use_cuts="fromfit",
    metadata_group="nnpdf31_process",
    theory={"from_": "fit"},
    theoryid={"from_": "theory"},
)
API.procs_index(**common_dict)

MultiIndex([('DIS NC',   'HERACOMBNCEP920', 108),
            ('DIS NC',   'HERACOMBNCEP920', 109),
            ('DIS NC',   'HERACOMBNCEP920', 110),
            ('DIS NC',   'HERACOMBNCEP920', 111),
            ('DIS NC',   'HERACOMBNCEP920', 112),
            ('DIS NC',   'HERACOMBNCEP920', 113),
            ('DIS NC',   'HERACOMBNCEP920', 114),
            ('DIS NC',   'HERACOMBNCEP920', 115),
            ('DIS NC',   'HERACOMBNCEP920', 116),
            ('DIS NC',   'HERACOMBNCEP920', 117),
            ...
            ('DIS CC', 'CHORUSNUPb_dw_ite', 596),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 597),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 598),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 599),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 600),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 601),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 602),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 604),
            ('DIS CC', 'CHORUSNUPb_dw_ite', 605),
            ('DIS CC', 'CHORUSNUPb

In [6]:
process_info.namelist

defaultdict(list,
            {'DIS NC': ['HERACOMBNCEP920', 'NMCPD_dw_ite'],
             'DIS CC': ['NTVNBDMNFe_dw_ite', 'CHORUSNUPb_dw_ite']})

In [7]:
process_info.sizes

defaultdict(list,
            {'HERACOMBNCEP920': 377,
             'NMCPD_dw_ite': 121,
             'NTVNBDMNFe_dw_ite': 37,
             'CHORUSNUPb_dw_ite': 416})

Compute delta_T_tilde (Eq. 3.37) and P_tilde (Eq. 3.38) of arXiv:2105.05114

In [8]:
# preds_onlyreplicas = preds.iloc[:, 2:].to_numpy()
# mean_prediction = np.mean(preds_onlyreplicas,axis=1)

# X = np.zeros((preds.shape[0],preds.shape[0]))
# for i in range(preds_onlyreplicas.shape[1]):
#     X += np.outer(
#         (preds_onlyreplicas[:, i] - mean_prediction), (preds_onlyreplicas[:, i] - mean_prediction)
#     )
# X *= 1 / preds_onlyreplicas.shape[1]

In [9]:
# pseudodata = API.read_pdf_pseudodata(**common_dict)
# dat_central = np.mean(
#     [i.pseudodata.reindex(preds.index.to_list()).to_numpy().flatten() for i in pseudodata],
#     axis=0,
# )

In [10]:
sum(preds_ht_cov_construction["theory_central"].index != dsindex)

0

In [11]:
# Theory covariance matrix
delta_pred = ht_coeff * (preds_ht_cov_construction["theory_central"] / kin_df['kin2'] ** 2 / ( 1 - kin_df['kin1'] )).to_numpy()
S = np.outer(delta_pred,delta_pred)
S = pd.DataFrame(S,index=dsindex,columns=dsindex)

# Experimental covariance matrix
C = API.groups_covmat_no_table(**common_dict)

In [12]:
# beta_tilde = np.sqrt(ht_coeff / 2) * np.array([1, -1])
# S_tilde = beta_tilde @ beta_tilde
# beta = [delta_pred, -delta_pred]
# S_hat = beta_tilde @ beta

# invcov = np.linalg.inv(C+S)

# ht_coeff_central = 0.0
# delta_T_tilde = S_hat @ invcov @ (dat_central - mean_prediction)
# P_tilde = S_hat.T @ invcov @ X @ invcov @ S_hat + (S_tilde - S_hat.T @ invcov @ S_hat)
# pred = ht_coeff_central + delta_T_tilde
# unc = np.sqrt(P_tilde)

In [166]:
fitpath = API.fit(fit=fitname).path
try:
    stored_covmat = pd.read_csv(
        fitpath / "tables/datacuts_theory_theorycovmatconfig_user_covmat.csv",
        sep="\t",
        encoding="utf-8",
        index_col=2,
        header=3,
        skip_blank_lines=False,
    )
except FileNotFoundError:
    stored_covmat = pd.read_csv(
            fitpath / "tables/datacuts_theory_theorycovmatconfig_theory_covmat_custom.csv",
            index_col=[0, 1, 2],
            header=[0, 1, 2], 
            sep="\t|,",
            engine="python",
            # converters={'id': np.int64}
        ).fillna(0)
    storedcovmat_index = pd.MultiIndex.from_tuples(
        [(aa, bb, np.int64(cc)) for aa, bb, cc in stored_covmat.index],
        names=['group', 'dataset', 'id']
    )
    stored_covmat = pd.DataFrame(stored_covmat.values, index=storedcovmat_index, columns=storedcovmat_index)
    stored_covmat = stored_covmat.reindex(C.index).T.reindex(C.index)
if np.allclose(S, stored_covmat):
    print(rf"Prediction for $\alpha_s$: {pred:.5f} ± {unc:.5f}")
else:
    print("Reconstructed theory covmat, S, is not the same as the stored covmat!")

In [172]:
np.allclose(S, stored_covmat)

True

In [168]:
S

group                                 DIS NC                                \
dataset                      HERACOMBNCEP920                                 
id                                       108       109       110       111   
group  dataset           id                                                  
DIS NC HERACOMBNCEP920   108        0.032029  0.032168  0.032261  0.032289   
                         109        0.032168  0.032307  0.032401  0.032429   
                         110        0.032261  0.032401  0.032496  0.032523   
                         111        0.032289  0.032429  0.032523  0.032551   
                         112        0.032270  0.032410  0.032504  0.032532   
...                                      ...       ...       ...       ...   
DIS CC CHORUSNUPb_dw_ite 601        0.000107  0.000108  0.000108  0.000108   
                         602        0.000056  0.000057  0.000057  0.000057   
                         604        0.000106  0.000106  0.000106  0.000107   
                         605        0.000042  0.000042  0.000042  0.000042   
                         606        0.000022  0.000022  0.000022  0.000022   

group                                                                 \
dataset                                                                
id                                 112       113       114       115   
group  dataset           id                                            
DIS NC HERACOMBNCEP920   108  0.032270  0.031387  0.029545  0.027721   
                         109  0.032410  0.031523  0.029674  0.027841   
                         110  0.032504  0.031615  0.029760  0.027922   
                         111  0.032532  0.031642  0.029785  0.027946   
                         112  0.032513  0.031623  0.029768  0.027929   
...                                ...       ...       ...       ...   
DIS CC CHORUSNUPb_dw_ite 601  0.000108  0.000105  0.000099  0.000093   
                         602  0.000057  0.000055  0.000052  0.000049   
                         604  0.000106  0.000104  0.000098  0.000091   
                         605  0.000042  0.000041  0.000039  0.000036   
                         606  0.000022  0.000021  0.000020  0.000019   

group                                             ...            DIS CC  \
dataset                                           ... CHORUSNUPb_dw_ite   
id                                 116       117  ...               596   
group  dataset           id                       ...                     
DIS NC HERACOMBNCEP920   108  0.025708  0.023829  ...      6.087842e-04   
                         109  0.025819  0.023933  ...      6.114255e-04   
                         110  0.025895  0.024003  ...      6.132081e-04   
                         111  0.025916  0.024023  ...      6.137240e-04   
                         112  0.025901  0.024009  ...      6.133657e-04   
...                                ...       ...  ...               ...   
DIS CC CHORUSNUPb_dw_ite 601  0.000086  0.000080  ...      2.040168e-06   
                         602  0.000045  0.000042  ...      1.071201e-06   
                         604  0.000085  0.000079  ...      2.009124e-06   
                         605  0.000034  0.000031  ...      7.954647e-07   
                         606  0.000017  0.000016  ...      4.111560e-07   

group                                                                   \
dataset                                                                  
id                                     597           598           599   
group  dataset           id                                              
DIS NC HERACOMBNCEP920   108  2.518271e-04  1.341826e-04  1.243875e-03   
                         109  2.529197e-04  1.347648e-04  1.249271e-03   
                         110  2.536571e-04  1.351577e-04  1.252913e-03   
                         111  2.538704e-04  1.352714e-04  1.253968e-03   
                         112  

In [169]:
stored_covmat

group                                 DIS NC                                \
dataset                      HERACOMBNCEP920                                 
id                                       108       109       110       111   
group  dataset           id                                                  
DIS NC HERACOMBNCEP920   108        0.032029  0.032168  0.032261  0.032289   
                         109        0.032168  0.032307  0.032401  0.032429   
                         110        0.032261  0.032401  0.032496  0.032523   
                         111        0.032289  0.032429  0.032523  0.032551   
                         112        0.032270  0.032410  0.032504  0.032532   
...                                      ...       ...       ...       ...   
DIS CC CHORUSNUPb_dw_ite 601        0.000107  0.000108  0.000108  0.000108   
                         602        0.000056  0.000057  0.000057  0.000057   
                         604        0.000106  0.000106  0.000106  0.000107   
                         605        0.000042  0.000042  0.000042  0.000042   
                         606        0.000022  0.000022  0.000022  0.000022   

group                                                                 \
dataset                                                                
id                                 112       113       114       115   
group  dataset           id                                            
DIS NC HERACOMBNCEP920   108  0.032270  0.031387  0.029545  0.027721   
                         109  0.032410  0.031523  0.029674  0.027841   
                         110  0.032504  0.031615  0.029760  0.027922   
                         111  0.032532  0.031642  0.029785  0.027946   
                         112  0.032513  0.031623  0.029768  0.027929   
...                                ...       ...       ...       ...   
DIS CC CHORUSNUPb_dw_ite 601  0.000108  0.000105  0.000099  0.000093   
                         602  0.000057  0.000055  0.000052  0.000049   
                         604  0.000106  0.000104  0.000098  0.000091   
                         605  0.000042  0.000041  0.000039  0.000036   
                         606  0.000022  0.000021  0.000020  0.000019   

group                                             ...            DIS CC  \
dataset                                           ... CHORUSNUPb_dw_ite   
id                                 116       117  ...               596   
group  dataset           id                       ...                     
DIS NC HERACOMBNCEP920   108  0.025708  0.023829  ...      6.087842e-04   
                         109  0.025819  0.023933  ...      6.114254e-04   
                         110  0.025895  0.024003  ...      6.132081e-04   
                         111  0.025916  0.024023  ...      6.137239e-04   
                         112  0.025901  0.024009  ...      6.133657e-04   
...                                ...       ...  ...               ...   
DIS CC CHORUSNUPb_dw_ite 601  0.000086  0.000080  ...      2.040168e-06   
                         602  0.000045  0.000042  ...      1.071201e-06   
                         604  0.000085  0.000079  ...      2.009124e-06   
                         605  0.000034  0.000031  ...      7.954647e-07   
                         606  0.000017  0.000016  ...      4.111560e-07   

group                                                                   \
dataset                                                                  
id                                     597           598           599   
group  dataset           id                                              
DIS NC HERACOMBNCEP920   108  2.518271e-04  1.341826e-04  1.243875e-03   
                         109  2.529197e-04  1.347648e-04  1.249271e-03   
                         110  2.536570e-04  1.351577e-04  1.252913e-03   
                         111  2.538704e-04  1.352714e-04  1.253968e-03   
                         112  